In [ ]:
import pandas as pd
import glob

# Alle meetpunten inladen vanuit Waterquality/Meetpunten
csv_files = glob.glob('Waterquality/Meetpunten/*.csv')
df_list = []
for file in csv_files:
    print(f"Laden van bestand: {file}")
    df = pd.read_csv(file)
    df_list.append(df)

# Samenvoegen in één DataFrame
df_all = pd.concat(df_list, ignore_index=True)
print(f"Totaal aantal meetpunten: {len(df_all)}")

# Exporteren naar CSV
output_path = 'all_meetpunten.csv'
df_all.to_csv(output_path, index=False)
print(f"Alle meetpunten opgeslagen in '{output_path}'")
